This architecture will be slightly different from language translation seq2seq. In translation model, the decoder output of last timestep will be feed into the model as decoder input for this timestep. People call it inference model. Here we will not make prediction based on the previous day prediction.

We will use past 400 days data to predict future 39 days, in alignment with the submission sample data.
In our case the encoder will be feed in (n_stores, t_timesteps, m_features). m_features will be the day info, reservation info, store info, weather info we use dummy variable for categorical features. 

The decoder would be feed in with the hidden state of encoder and also decoder_input(which are the features of dates info, weather info )

encoder input data:
one sample(store) would be like

| time| v1 | v2 | v3 | v4 | v5 |
| --- | --- | --- | --- | --- | ---|
|t=1|rain|genre1|visitors 11| reservation 5|
|t=2|sunny|genre1|visitors 20| reservation 5|
|...|
|t=400|xxx|xxx|xxxx|xxxxx|

decoder input data:

| time| v1 | v2 | v3 |
| --- | --- | --- | --- |
|t=400+1|genre1|region1|date info|
|...|
|t=400+39|genre1|region1|date info|


decoder target data:
the same sample(store)

| time| target | 
| --- | ------ |
|t=400+1|visitors|
|...|
|t=400+39|visitors|

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/finalproject/master_final.csv
/kaggle/input/master-v3/master_v3.csv
/kaggle/input/recruit-restaurant-visitor-forecasting/hpg_store_info.csv
/kaggle/input/recruit-restaurant-visitor-forecasting/air_reserve.csv
/kaggle/input/recruit-restaurant-visitor-forecasting/date_info.csv
/kaggle/input/recruit-restaurant-visitor-forecasting/sample_submission.csv
/kaggle/input/recruit-restaurant-visitor-forecasting/store_id_relation.csv
/kaggle/input/recruit-restaurant-visitor-forecasting/air_visit_data.csv
/kaggle/input/recruit-restaurant-visitor-forecasting/hpg_reserve.csv
/kaggle/input/recruit-restaurant-visitor-forecasting/air_store_info.csv


In [2]:
import pandas as pd
import numpy as np
import random
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.max_seq_items = 2000

In [3]:
full = pd.read_csv('/kaggle/input/finalproject/master_final.csv')
sample_submission = pd.read_csv('/kaggle/input/recruit-restaurant-visitor-forecasting/sample_submission.csv')

# transform submission test data
samp_sub = sample_submission.copy()
samp_sub['air_store_id'] = samp_sub['id'].apply(lambda x: str(x)[:-11])
samp_sub['calendar_date'] = samp_sub['id'].apply(lambda x: str(x)[-10:])

In [4]:
full['calendar_date'] = pd.to_datetime(full['calendar_date'])
samp_sub['calendar_date'] = pd.to_datetime(samp_sub['calendar_date']).dt.date

In [5]:
full.head(2)

,air_store_id,calendar_date,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,reserve_visitors_air,hpg_store_id,reserve_visitors_hpg,visitors,day,month,year,dayofweek,dayofyear,earlist_open_date,closed_flag,lookback_visitors_avg_store,lookback_visitors_avg_genre,lookback_visitors_avg_area,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,avg_wind_speed,avg_humidity,day_after_holiday_flg,day_before_holiday_flg,week_of_month,open_days,lookback_visitors_std_store,lookback_visitors_std_genre,lookback_visitors_std_area
0,air_00a91d42b08b08d9,2016-01-01,Friday,1,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN,hpg_fbe603376b5980fc,NaN,NaN,1,1,2016,4,1,2016-07-01,2,26.08,23.282404,24.763514,7.5,12.1,3.6,0.0,9.0,2.6,45.0,0.0,0.0,one,0,8.406347,13.85957,14.56454
1,air_00a91d42b08b08d9,2016-01-02,Saturday,1,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN,hpg_fbe603376b5980fc,NaN,NaN,2,1,2016,5,2,2016-07-01,2,26.08,23.282404,24.763514,7.3,13.4,2.6,0.0,7.5,1.9,51.0,0.0,0.0,one,0,8.406347,13.85957,14.56454


In [6]:
full.tail(2)

,air_store_id,calendar_date,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,reserve_visitors_air,hpg_store_id,reserve_visitors_hpg,visitors,day,month,year,dayofweek,dayofyear,earlist_open_date,closed_flag,lookback_visitors_avg_store,lookback_visitors_avg_genre,lookback_visitors_avg_area,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,avg_wind_speed,avg_humidity,day_after_holiday_flg,day_before_holiday_flg,week_of_month,open_days,lookback_visitors_std_store,lookback_visitors_std_genre,lookback_visitors_std_area
428591,air_fff68b929994bfbd,2017-05-30,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,NaN,NaN,NaN,NaN,30,5,2017,1,150,2016-07-01,1,5.0,13.060498,15.125000,24.0,31.5,17.6,0.0,11.9,1.0,70.000002,0.0,0.0,four/five,333,2.629956,10.344121,4.274091
428592,air_fff68b929994bfbd,2017-05-31,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,NaN,NaN,NaN,NaN,31,5,2017,2,151,2016-07-01,1,4.0,10.864111,14.217391,24.0,29.6,20.5,0.0,3.4,1.2,76.000002,0.0,0.0,four/five,334,3.774917,12.448389,4.450789


In [7]:
full.columns

Index(['air_store_id', 'calendar_date', 'day_of_week', 'holiday_flg',
       'air_genre_name', 'air_area_name', 'latitude', 'longitude',
       'reserve_visitors_air', 'hpg_store_id', 'reserve_visitors_hpg',
       'visitors', 'day', 'month', 'year', 'dayofweek', 'dayofyear',
       'earlist_open_date', 'closed_flag', 'lookback_visitors_avg_store',
       'lookback_visitors_avg_genre', 'lookback_visitors_avg_area',
       'avg_temperature', 'high_temperature', 'low_temperature',
       'precipitation', 'hours_sunlight', 'avg_wind_speed', 'avg_humidity',
       'day_after_holiday_flg', 'day_before_holiday_flg', 'week_of_month',
       'open_days', 'lookback_visitors_std_store',
       'lookback_visitors_std_genre', 'lookback_visitors_std_area'],
      dtype='object')

In [8]:
print('full table has',full.air_store_id.nunique(),'stores' )
print('full table calendar period', full.calendar_date.min(), '--', full.calendar_date.max())
print('Sample submission file has',samp_sub.air_store_id.nunique(),'stores' )
print('Sample submission prediction period is', samp_sub.calendar_date.min(), '--', samp_sub.calendar_date.max())
print('On total we need to predict %s days.'%(int((samp_sub.calendar_date.max()-samp_sub.calendar_date.min()).days)+1))
print('Therefore the decoder output timestep would be 39 days, but we can arbitrarily define the encoder input timestep')

full table has 829 stores
full table calendar period 2016-01-01 00:00:00 -- 2017-05-31 00:00:00
Sample submission file has 821 stores
Sample submission prediction period is 2017-04-23 -- 2017-05-31
On total we need to predict 39 days.
Therefore the decoder output timestep would be 39 days, but we can arbitrarily define the encoder input timestep


In [9]:
pd.to_datetime('2016-01-01') - pd.to_datetime('2017-04-22') # 478 days that we have visitors info

Timedelta('-477 days +00:00:00')

# Define train_validation split date

A walk forward split for train and validation dataset, details can be found in the following links
[https://github.com/JEddy92/TimeSeries_Seq2Seq/blob/master/notebooks/TS_Seq2Seq_Intro.ipynb]

1. Train encoding period (400 days)
2. Train decoding period (train targets, 39 days)
3. Validation encoding period (400 days)
4. Validation decoding period (validation targets, 39 days)

In [10]:
from datetime import timedelta
import datetime

In [11]:
LOOKBACK_DAYS = 400
PRED_DAYS = 39

In [12]:
# create a function that will give us the time period for training model
# in default we would use 400 days as train encoding, because we have in total 477 days that have visitors info, 
# and we need to shift to accomondate the validation period.
def calculate_date_split(data_start_date ='2016-01-01', validation_end_date= '2017-04-22', t = 400, pred_days=39):
    val_pred_end = pd.to_datetime(validation_end_date)
    val_pred_start = pd.to_datetime(validation_end_date) - timedelta(days= pred_days - 1)
    
    val_enc_end = val_pred_start - timedelta(days = 1)
    val_enc_start = val_enc_end - timedelta(days = t - 1)
    
    train_pred_end = val_enc_end
    train_pred_start = train_pred_end - timedelta(days= pred_days - 1)
    
    train_enc_end = train_pred_start - timedelta(days = 1)
    train_enc_start = train_enc_end - timedelta(days = t - 1)
    
    if train_enc_start < pd.to_datetime(data_start_date):
        print('the tranining encoding period out of date')
    
    test_pred_start = val_pred_end + timedelta(days = 1) 
    test_pred_end = test_pred_start + timedelta(days = pred_days -1)
    
    test_enc_end = test_pred_start - timedelta(days = 1)
    test_enc_start = test_enc_end - timedelta(days = t - 1)

    return test_enc_start, test_enc_end, test_pred_start, test_pred_end,\
           train_enc_start, train_enc_end, train_pred_start,train_pred_end, \
           val_enc_start, val_enc_end, val_pred_start, val_pred_end

In [13]:
test_enc_start, test_enc_end, test_pred_start, test_pred_end, \
train_enc_start, train_enc_end, train_pred_start,train_pred_end, \
val_enc_start, val_enc_end, val_pred_start, val_pred_end = calculate_date_split(t = LOOKBACK_DAYS, pred_days = PRED_DAYS)

In [14]:
print('Train encoding:', train_enc_start, '-', train_enc_end)
print('Train prediction:', train_pred_start, '-', train_pred_end, '\n')
print('Val encoding:', val_enc_start, '-', val_enc_end)
print('Val prediction:', val_pred_start, '-', val_pred_end,'\n')
print('Test encoding:', test_enc_start, '-', test_enc_end)
print('Test prediction:', test_pred_start, '-', test_pred_end)

print('\nEncoding interval:', (train_pred_start - train_enc_start).days)
print('Prediction interval:', (train_pred_end - train_enc_end).days)

Train encoding: 2016-01-01 00:00:00 - 2017-02-03 00:00:00
Train prediction: 2017-02-04 00:00:00 - 2017-03-14 00:00:00 

Val encoding: 2016-02-09 00:00:00 - 2017-03-14 00:00:00
Val prediction: 2017-03-15 00:00:00 - 2017-04-22 00:00:00 

Test encoding: 2016-03-19 00:00:00 - 2017-04-22 00:00:00
Test prediction: 2017-04-23 00:00:00 - 2017-05-31 00:00:00

Encoding interval: 400
Prediction interval: 39


# Feature Selection And Normalization
reference from[https://www.kaggle.com/ievgenvp/lstm-encoder-decoder-via-keras-lb-0-5]

1. nominal feature label/or ??onehot encoding,,,use numeric label encoder...see what we would got..
2. minmax scaling selected features that would go into the model

### step1

In [15]:
full.loc[full['week_of_month'] == 'one','week_of_month'] = 1
full.loc[full['week_of_month'] == 'two','week_of_month'] = 2
full.loc[full['week_of_month'] == 'three','week_of_month'] = 3
full.loc[full['week_of_month'] == 'four/five','week_of_month'] = 4

In [16]:
# one hot encoding get dummy variables
full = pd.concat([full,pd.get_dummies(full['air_genre_name'], prefix = 'genre').astype(float)], axis = 1)
full = pd.concat([full,pd.get_dummies(full['air_area_name'], prefix = 'area').astype(float)], axis = 1)

In [17]:
# label encoder
#from sklearn.preprocessing import LabelEncoder
# Genre name
#le_genre = LabelEncoder()
#le_genre.fit(full['air_genre_name'])
#full['air_genre_name_ix'] = le_genre.transform(full['air_genre_name'])

# Area name
#le_area = LabelEncoder()
#le_area.fit(full['air_area_name'])
#full['air_area_name_ix'] = le_area.transform(full['air_area_name'])

In [18]:
full.iloc[:,2:].columns.get_loc("lookback_visitors_std_area")

33

In [19]:
full.dtypes

air_store_id                                                  object
calendar_date                                         datetime64[ns]
day_of_week                                                   object
holiday_flg                                                    int64
air_genre_name                                                object
air_area_name                                                 object
latitude                                                     float64
longitude                                                    float64
reserve_visitors_air                                         float64
hpg_store_id                                                  object
reserve_visitors_hpg                                         float64
visitors                                                     float64
day                                                            int64
month                                                          int64
year                              

In [20]:
full.columns 

Index(['air_store_id', 'calendar_date', 'day_of_week', 'holiday_flg',
       'air_genre_name', 'air_area_name', 'latitude', 'longitude',
       'reserve_visitors_air', 'hpg_store_id', 'reserve_visitors_hpg',
       'visitors', 'day', 'month', 'year', 'dayofweek', 'dayofyear',
       'earlist_open_date', 'closed_flag', 'lookback_visitors_avg_store',
       'lookback_visitors_avg_genre', 'lookback_visitors_avg_area',
       'avg_temperature', 'high_temperature', 'low_temperature',
       'precipitation', 'hours_sunlight', 'avg_wind_speed', 'avg_humidity',
       'day_after_holiday_flg', 'day_before_holiday_flg', 'week_of_month',
       'open_days', 'lookback_visitors_std_store',
       'lookback_visitors_std_genre', 'lookback_visitors_std_area',
       'genre_Asian', 'genre_Bar/Cocktail', 'genre_Cafe/Sweets',
       'genre_Creative cuisine', 'genre_Dining bar',
       'genre_International cuisine', 'genre_Italian/French', 'genre_Izakaya',
       'genre_Japanese food', 'genre_Karaoke/Part

In [21]:
# select the features that you want
# we keep air_store_id and calendar_date columns to transform input data shape later

# we purposefully put unstationary features in the end of the list
# by stationary I mean genre,area,day of year etc.. those features are certainly known before prediction.
# because we will use stationary features as decoder input later

selected_cols = ['air_store_id', 'calendar_date', 'visitors', 'holiday_flg',
       'latitude', 'longitude',
       'day', 'month', 'year', 'dayofweek', 'dayofyear',
       
       'lookback_visitors_avg_store',
       'lookback_visitors_avg_genre', 'lookback_visitors_avg_area',
       'avg_temperature', 'high_temperature', 'low_temperature',
       'precipitation', 'hours_sunlight', 'avg_wind_speed', 'avg_humidity',
       'day_after_holiday_flg', 'day_before_holiday_flg', 'week_of_month',
       'open_days', 'lookback_visitors_std_store',
       'lookback_visitors_std_genre', 'lookback_visitors_std_area',
                 
       'genre_Asian', 'genre_Bar/Cocktail', 'genre_Cafe/Sweets',
       'genre_Creative cuisine', 'genre_Dining bar',
       'genre_International cuisine', 'genre_Italian/French', 'genre_Izakaya',
       'genre_Japanese food', 'genre_Karaoke/Party',
       'genre_Okonomiyaki/Monja/Teppanyaki', 'genre_Other',
       'genre_Western food', 'genre_Yakiniku/Korean food',
       'area_Fukuoka-ken Fukuoka-shi Daimyō',
       'area_Fukuoka-ken Fukuoka-shi Hakata Ekimae',
       'area_Fukuoka-ken Fukuoka-shi Imaizumi',
       'area_Fukuoka-ken Fukuoka-shi Momochi',
       'area_Fukuoka-ken Fukuoka-shi Shiobaru',
       'area_Fukuoka-ken Fukuoka-shi Takatori',
       'area_Fukuoka-ken Fukuoka-shi Tenjin',
       'area_Fukuoka-ken Fukuoka-shi Torikai',
       'area_Fukuoka-ken Itoshima-shi Maebarunishi',
       'area_Fukuoka-ken Kitakyūshū-shi Konyamachi',
       'area_Fukuoka-ken Kitakyūshū-shi None',
       'area_Fukuoka-ken Kitakyūshū-shi Ōtemachi',
       'area_Fukuoka-ken Kurume-shi Jōnanmachi',
       'area_Fukuoka-ken Yame-shi Motomachi',
       'area_Hiroshima-ken Fukuyama-shi Higashisakuramachi',
       'area_Hiroshima-ken Hiroshima-shi Kokutaijimachi',
       'area_Hiroshima-ken Hiroshima-shi Nagarekawachō',
       'area_Hokkaidō Abashiri-shi Minami 6 Jōhigashi',
       'area_Hokkaidō Asahikawa-shi 6 Jōdōri',
       'area_Hokkaidō Katō-gun Motomachi',
       'area_Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō',
       'area_Hokkaidō Sapporo-shi Kita 24 Jōnishi',
       'area_Hokkaidō Sapporo-shi Kotoni 2 Jō',
       'area_Hokkaidō Sapporo-shi Minami 3 Jōnishi',
       'area_Hyōgo-ken Amagasaki-shi Higashinanamatsuchō',
       'area_Hyōgo-ken Himeji-shi Yasuda',
       'area_Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike',
       'area_Hyōgo-ken Kōbe-shi Kumoidōri',
       'area_Hyōgo-ken Kōbe-shi Motomachidōri',
       'area_Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi',
       'area_Hyōgo-ken Nishinomiya-shi Rokutanjichō',
       'area_Hyōgo-ken Takarazuka-shi Tōyōchō',
       'area_Miyagi-ken Sendai-shi Kamisugi',
       'area_Niigata-ken Kashiwazaki-shi Chūōchō',
       'area_Niigata-ken Nagaoka-shi Ōtedōri',
       'area_Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō',
       'area_Niigata-ken Niigata-shi Teraohigashi',
       'area_Shizuoka-ken Hamamatsu-shi Motoshirochō',
       'area_Shizuoka-ken Numazu-shi Miyukichō',
       'area_Shizuoka-ken Shizuoka-shi Ōtemachi',
       'area_Tōkyō-to Adachi-ku Chūōhonchō', 'area_Tōkyō-to Bunkyō-ku Kasuga',
       'area_Tōkyō-to Bunkyō-ku Sendagi',
       'area_Tōkyō-to Chiyoda-ku Kanda Jinbōchō',
       'area_Tōkyō-to Chiyoda-ku Kudanminami', 'area_Tōkyō-to Chūō-ku Ginza',
       'area_Tōkyō-to Chūō-ku Tsukiji', 'area_Tōkyō-to Edogawa-ku Chūō',
       'area_Tōkyō-to Fuchū-shi Miyanishichō',
       'area_Tōkyō-to Itabashi-ku Itabashi',
       'area_Tōkyō-to Katsushika-ku Tateishi',
       'area_Tōkyō-to Kita-ku Ōjihonchō', 'area_Tōkyō-to Koganei-shi Honchō',
       'area_Tōkyō-to Kōtō-ku Kameido', 'area_Tōkyō-to Kōtō-ku Tomioka',
       'area_Tōkyō-to Kōtō-ku Tōyō', 'area_Tōkyō-to Machida-shi Morino',
       'area_Tōkyō-to Meguro-ku Jiyūgaoka',
       'area_Tōkyō-to Meguro-ku Kamimeguro', 'area_Tōkyō-to Meguro-ku Takaban',
       'area_Tōkyō-to Minato-ku Akasaka', 'area_Tōkyō-to Minato-ku Roppongi',
       'area_Tōkyō-to Minato-ku Shibakōen',
       'area_Tōkyō-to Musashino-shi Kichijōji Honchō',
       'area_Tōkyō-to Musashino-shi Midorichō',
       'area_Tōkyō-to Nakano-ku Nakano',
       'area_Tōkyō-to Nerima-ku Toyotamakita',
       'area_Tōkyō-to Setagaya-ku Kitazawa',
       'area_Tōkyō-to Setagaya-ku Okusawa',
       'area_Tōkyō-to Setagaya-ku Sangenjaya',
       'area_Tōkyō-to Setagaya-ku Setagaya',
       'area_Tōkyō-to Shibuya-ku Dōgenzaka',
       'area_Tōkyō-to Shibuya-ku Higashi', 'area_Tōkyō-to Shibuya-ku Jingūmae',
       'area_Tōkyō-to Shibuya-ku Shibuya',
       'area_Tōkyō-to Shinagawa-ku Higashigotanda',
       'area_Tōkyō-to Shinagawa-ku Hiromachi',
       'area_Tōkyō-to Shinjuku-ku Kabukichō',
       'area_Tōkyō-to Shinjuku-ku Nishishinjuku',
       'area_Tōkyō-to Suginami-ku Asagayaminami',
       'area_Tōkyō-to Tachikawa-shi Izumichō',
       'area_Tōkyō-to Taitō-ku Asakusa', 'area_Tōkyō-to Taitō-ku Higashiueno',
       'area_Tōkyō-to Taitō-ku Kuramae',
       'area_Tōkyō-to Toshima-ku Higashiikebukuro',
       'area_Tōkyō-to Toshima-ku Mejiro',
       'area_Tōkyō-to Toshima-ku Minamiikebukuro',
       'area_Tōkyō-to Toshima-ku Nishiikebukuro',
       'area_Tōkyō-to Toshima-ku Sugamo', 'area_Tōkyō-to Ōta-ku Kamata',
       'area_Tōkyō-to Ōta-ku Kamiikedai',
       'area_Ōsaka-fu Higashiōsaka-shi Aramotokita',
       'area_Ōsaka-fu Neyagawa-shi Honmachi',
       'area_Ōsaka-fu Sakai-shi Minamikawaramachi',
       'area_Ōsaka-fu Suita-shi Izumichō',
       'area_Ōsaka-fu Ōsaka-shi Fuminosato',
       'area_Ōsaka-fu Ōsaka-shi Kyōmachibori',
       'area_Ōsaka-fu Ōsaka-shi Kyūtarōmachi',
       'area_Ōsaka-fu Ōsaka-shi Nakanochō',
       'area_Ōsaka-fu Ōsaka-shi Nanbasennichimae',
       'area_Ōsaka-fu Ōsaka-shi Shinmachi', 'area_Ōsaka-fu Ōsaka-shi Ōgimachi',
       'area_Ōsaka-fu Ōsaka-shi Ōhiraki',
                 
        'closed_flag', 'reserve_visitors_air', 'reserve_visitors_hpg'
        
        ]
df_to_reshape = full[selected_cols]
df_to_reshape.replace(np.nan, 0, inplace = True)# replace Missing value with 0

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [22]:
# transform numeric value to float type..because float type is a must for NN??
df_to_reshape['holiday_flg'] = df_to_reshape.holiday_flg.astype('float64')
#df_to_reshape['air_genre_name_ix'] = df_to_reshape.air_genre_name_ix.astype('float64')
#df_to_reshape['air_area_name_ix'] = df_to_reshape.air_area_name_ix.astype('float64')
df_to_reshape['day'] = df_to_reshape.day.astype('float64')
df_to_reshape['month'] = df_to_reshape.month.astype('float64')
df_to_reshape['year'] = df_to_reshape.year.astype('float64')
df_to_reshape['dayofweek'] = df_to_reshape.dayofweek.astype('float64')
df_to_reshape['dayofyear'] = df_to_reshape.dayofyear.astype('float64')
df_to_reshape['day_after_holiday_flg'] = df_to_reshape.day_after_holiday_flg.astype('float64')
df_to_reshape['day_before_holiday_flg'] = df_to_reshape.day_before_holiday_flg.astype('float64')
df_to_reshape['week_of_month'] = df_to_reshape.week_of_month.astype('float64')
df_to_reshape['open_days'] = df_to_reshape.open_days.astype('float64')
df_to_reshape['closed_flag'] = df_to_reshape.closed_flag.astype('float64')

df_to_reshape.dtypes

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/s

air_store_id                                                  object
calendar_date                                         datetime64[ns]
visitors                                                     float64
holiday_flg                                                  float64
latitude                                                     float64
longitude                                                    float64
day                                                          float64
month                                                        float64
year                                                         float64
dayofweek                                                    float64
dayofyear                                                    float64
lookback_visitors_avg_store                                  float64
lookback_visitors_avg_genre                                  float64
lookback_visitors_avg_area                                   float64
avg_temperature                   

### step2

In [23]:
from sklearn.preprocessing import MinMaxScaler
columns_list_toscale = selected_cols[3:] 
# we do not scale 'visitors' 

In [24]:
# bound all numerical values between -1 and 1
# note: to avoid data leakage 'fit' should be made on traid data and 'transform' on train and test data
# in this case all data in test set is taken from train set, thus fit/transform on all data 
df_scaled = df_to_reshape.copy()
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(df_scaled[columns_list_toscale])
df_scaled[columns_list_toscale] = scaler.transform(df_scaled[columns_list_toscale])
df_scaled.sort_values(by=['air_store_id','calendar_date'], inplace = True)

#scaler.inverse_transform(scaled_data) # for inverse transformation

In [25]:
# we use log(1+x) to transform visitors
df_scaled['visitors'] = np.log1p(df_scaled['visitors'])

#Y = np.log1p(Y)
#back = np.expm1(Y)

In [26]:
df_scaled.head()

,air_store_id,calendar_date,visitors,holiday_flg,latitude,longitude,day,month,year,dayofweek,dayofyear,lookback_visitors_avg_store,lookback_visitors_avg_genre,lookback_visitors_avg_area,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,avg_wind_speed,avg_humidity,day_after_holiday_flg,day_before_holiday_flg,week_of_month,open_days,lookback_visitors_std_store,lookback_visitors_std_genre,lookback_visitors_std_area,genre_Asian,genre_Bar/Cocktail,genre_Cafe/Sweets,genre_Creative cuisine,genre_Dining bar,genre_International cuisine,genre_Italian/French,genre_Izakaya,genre_Japanese food,genre_Karaoke/Party,genre_Okonomiyaki/Monja/Teppanyaki,genre_Other,genre_Western food,genre_Yakiniku/Korean food,area_Fukuoka-ken Fukuoka-shi Daimyō,area_Fukuoka-ken Fukuoka-shi Hakata Ekimae,area_Fukuoka-ken Fukuoka-shi Imaizumi,area_Fukuoka-ken Fukuoka-shi Momochi,area_Fukuoka-ken Fukuoka-shi Shiobaru,area_Fukuoka-ken Fukuoka-shi Takatori,area_Fukuoka-ken Fukuoka-shi Tenjin,area_Fukuoka-ken Fukuoka-shi Torikai,area_Fukuoka-ken Itoshima-shi Maebarunishi,area_Fukuoka-ken Kitakyūshū-shi Konyamachi,area_Fukuoka-ken Kitakyūshū-shi None,area_Fukuoka-ken Kitakyūshū-shi Ōtemachi,area_Fukuoka-ken Kurume-shi Jōnanmachi,area_Fukuoka-ken Yame-shi Motomachi,area_Hiroshima-ken Fukuyama-shi Higashisakuramachi,area_Hiroshima-ken Hiroshima-shi Kokutaijimachi,area_Hiroshima-ken Hiroshima-shi Nagarekawachō,area_Hokkaidō Abashiri-shi Minami 6 Jōhigashi,area_Hokkaidō Asahikawa-shi 6 Jōdōri,area_Hokkaidō Katō-gun Motomachi,area_Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō,area_Hokkaidō Sapporo-shi Kita 24 Jōnishi,area_Hokkaidō Sapporo-shi Kotoni 2 Jō,area_Hokkaidō Sapporo-shi Minami 3 Jōnishi,area_Hyōgo-ken Amagasaki-shi Higashinanamatsuchō,area_Hyōgo-ken Himeji-shi Yasuda,area_Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,area_Hyōgo-ken Kōbe-shi Kumoidōri,area_Hyōgo-ken Kōbe-shi Motomachidōri,area_Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi,area_Hyōgo-ken Nishinomiya-shi Rokutanjichō,area_Hyōgo-ken Takarazuka-shi Tōyōchō,area_Miyagi-ken Sendai-shi Kamisugi,area_Niigata-ken Kashiwazaki-shi Chūōchō,area_Niigata-ken Nagaoka-shi Ōtedōri,area_Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,area_Niigata-ken Niigata-shi Teraohigashi,area_Shizuoka-ken Hamamatsu-shi Motoshirochō,area_Shizuoka-ken Numazu-shi Miyukichō,area_Shizuoka-ken Shizuoka-shi Ōtemachi,area_Tōkyō-to Adachi-ku Chūōhonchō,area_Tōkyō-to Bunkyō-ku Kasuga,area_Tōkyō-to Bunkyō-ku Sendagi,area_Tōkyō-to Chiyoda-ku Kanda Jinbōchō,area_Tōkyō-to Chiyoda-ku Kudanminami,area_Tōkyō-to Chūō-ku Ginza,area_Tōkyō-to Chūō-ku Tsukiji,area_Tōkyō-to Edogawa-ku Chūō,area_Tōkyō-to Fuchū-shi Miyanishichō,area_Tōkyō-to Itabashi-ku Itabashi,area_Tōkyō-to Katsushika-ku Tateishi,area_Tōkyō-to Kita-ku Ōjihonchō,area_Tōkyō-to Koganei-shi Honchō,area_Tōkyō-to Kōtō-ku Kameido,area_Tōkyō-to Kōtō-ku Tomioka,area_Tōkyō-to Kōtō-ku Tōyō,area_Tōkyō-to Machida-shi Morino,area_Tōkyō-to Meguro-ku Jiyūgaoka,area_Tōkyō-to Meguro-ku Kamimeguro,area_Tōkyō-to Meguro-ku Takaban,area_Tōkyō-to Minato-ku Akasaka,area_Tōkyō-to Minato-ku Roppongi,area_Tōkyō-to Minato-ku Shibakōen,area_Tōkyō-to Musashino-shi Kichijōji Honchō,area_Tōkyō-to Musashino-shi Midorichō,area_Tōkyō-to Nakano-ku Nakano,area_Tōkyō-to Nerima-ku Toyotamakita,area_Tōkyō-to Setagaya-ku Kitazawa,area_Tōkyō-to Setagaya-ku Okusawa,area_Tōkyō-to Setagaya-ku Sangenjaya,area_Tōkyō-to Setagaya-ku Setagaya,area_Tōkyō-to Shibuya-ku Dōgenzaka,area_Tōkyō-to Shibuya-ku Higashi,area_Tōkyō-to Shibuya-ku Jingūmae,area_Tōkyō-to Shibuya-ku Shibuya,area_Tōkyō-to Shinagawa-ku Higashigotanda,area_Tōkyō-to Shinagawa-ku Hiromachi,area_Tōkyō-to Shinjuku-ku Kabukichō,area_Tōkyō-to Shinjuku-ku Nishishinjuku,area_Tōkyō-to Suginami-ku Asagayaminami,area_Tōkyō-to Tachikawa-shi Izumichō,area_Tōkyō-to Taitō-ku Asakusa,area_Tōkyō-to Taitō-ku Higashiueno,area_Tōkyō-to Taitō-ku Kuramae,area_Tōkyō-to Toshima-ku Higashiikebukuro,area_Tōkyō-to Toshima-ku Mejiro,area_Tōkyō-to Toshima-ku Minamiikebukuro,area_Tō

In [27]:
df_scaled.tail()

,air_store_id,calendar_date,visitors,holiday_flg,latitude,longitude,day,month,year,dayofweek,dayofyear,lookback_visitors_avg_store,lookback_visitors_avg_genre,lookback_visitors_avg_area,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,avg_wind_speed,avg_humidity,day_after_holiday_flg,day_before_holiday_flg,week_of_month,open_days,lookback_visitors_std_store,lookback_visitors_std_genre,lookback_visitors_std_area,genre_Asian,genre_Bar/Cocktail,genre_Cafe/Sweets,genre_Creative cuisine,genre_Dining bar,genre_International cuisine,genre_Italian/French,genre_Izakaya,genre_Japanese food,genre_Karaoke/Party,genre_Okonomiyaki/Monja/Teppanyaki,genre_Other,genre_Western food,genre_Yakiniku/Korean food,area_Fukuoka-ken Fukuoka-shi Daimyō,area_Fukuoka-ken Fukuoka-shi Hakata Ekimae,area_Fukuoka-ken Fukuoka-shi Imaizumi,area_Fukuoka-ken Fukuoka-shi Momochi,area_Fukuoka-ken Fukuoka-shi Shiobaru,area_Fukuoka-ken Fukuoka-shi Takatori,area_Fukuoka-ken Fukuoka-shi Tenjin,area_Fukuoka-ken Fukuoka-shi Torikai,area_Fukuoka-ken Itoshima-shi Maebarunishi,area_Fukuoka-ken Kitakyūshū-shi Konyamachi,area_Fukuoka-ken Kitakyūshū-shi None,area_Fukuoka-ken Kitakyūshū-shi Ōtemachi,area_Fukuoka-ken Kurume-shi Jōnanmachi,area_Fukuoka-ken Yame-shi Motomachi,area_Hiroshima-ken Fukuyama-shi Higashisakuramachi,area_Hiroshima-ken Hiroshima-shi Kokutaijimachi,area_Hiroshima-ken Hiroshima-shi Nagarekawachō,area_Hokkaidō Abashiri-shi Minami 6 Jōhigashi,area_Hokkaidō Asahikawa-shi 6 Jōdōri,area_Hokkaidō Katō-gun Motomachi,area_Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō,area_Hokkaidō Sapporo-shi Kita 24 Jōnishi,area_Hokkaidō Sapporo-shi Kotoni 2 Jō,area_Hokkaidō Sapporo-shi Minami 3 Jōnishi,area_Hyōgo-ken Amagasaki-shi Higashinanamatsuchō,area_Hyōgo-ken Himeji-shi Yasuda,area_Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,area_Hyōgo-ken Kōbe-shi Kumoidōri,area_Hyōgo-ken Kōbe-shi Motomachidōri,area_Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi,area_Hyōgo-ken Nishinomiya-shi Rokutanjichō,area_Hyōgo-ken Takarazuka-shi Tōyōchō,area_Miyagi-ken Sendai-shi Kamisugi,area_Niigata-ken Kashiwazaki-shi Chūōchō,area_Niigata-ken Nagaoka-shi Ōtedōri,area_Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,area_Niigata-ken Niigata-shi Teraohigashi,area_Shizuoka-ken Hamamatsu-shi Motoshirochō,area_Shizuoka-ken Numazu-shi Miyukichō,area_Shizuoka-ken Shizuoka-shi Ōtemachi,area_Tōkyō-to Adachi-ku Chūōhonchō,area_Tōkyō-to Bunkyō-ku Kasuga,area_Tōkyō-to Bunkyō-ku Sendagi,area_Tōkyō-to Chiyoda-ku Kanda Jinbōchō,area_Tōkyō-to Chiyoda-ku Kudanminami,area_Tōkyō-to Chūō-ku Ginza,area_Tōkyō-to Chūō-ku Tsukiji,area_Tōkyō-to Edogawa-ku Chūō,area_Tōkyō-to Fuchū-shi Miyanishichō,area_Tōkyō-to Itabashi-ku Itabashi,area_Tōkyō-to Katsushika-ku Tateishi,area_Tōkyō-to Kita-ku Ōjihonchō,area_Tōkyō-to Koganei-shi Honchō,area_Tōkyō-to Kōtō-ku Kameido,area_Tōkyō-to Kōtō-ku Tomioka,area_Tōkyō-to Kōtō-ku Tōyō,area_Tōkyō-to Machida-shi Morino,area_Tōkyō-to Meguro-ku Jiyūgaoka,area_Tōkyō-to Meguro-ku Kamimeguro,area_Tōkyō-to Meguro-ku Takaban,area_Tōkyō-to Minato-ku Akasaka,area_Tōkyō-to Minato-ku Roppongi,area_Tōkyō-to Minato-ku Shibakōen,area_Tōkyō-to Musashino-shi Kichijōji Honchō,area_Tōkyō-to Musashino-shi Midorichō,area_Tōkyō-to Nakano-ku Nakano,area_Tōkyō-to Nerima-ku Toyotamakita,area_Tōkyō-to Setagaya-ku Kitazawa,area_Tōkyō-to Setagaya-ku Okusawa,area_Tōkyō-to Setagaya-ku Sangenjaya,area_Tōkyō-to Setagaya-ku Setagaya,area_Tōkyō-to Shibuya-ku Dōgenzaka,area_Tōkyō-to Shibuya-ku Higashi,area_Tōkyō-to Shibuya-ku Jingūmae,area_Tōkyō-to Shibuya-ku Shibuya,area_Tōkyō-to Shinagawa-ku Higashigotanda,area_Tōkyō-to Shinagawa-ku Hiromachi,area_Tōkyō-to Shinjuku-ku Kabukichō,area_Tōkyō-to Shinjuku-ku Nishishinjuku,area_Tōkyō-to Suginami-ku Asagayaminami,area_Tōkyō-to Tachikawa-shi Izumichō,area_Tōkyō-to Taitō-ku Asakusa,area_Tōkyō-to Taitō-ku Higashiueno,area_Tōkyō-to Taitō-ku Kuramae,area_Tōkyō-to Toshima-ku Higashiikebukuro,area_Tōkyō-to Toshima-ku Mejiro,area_Tōkyō-to Toshima-ku Minamiikebukuro,area_Tō

# Training and Validation Input Data Format Preparation

LSTM cell accept input in shape(n_samples, t_timesteps, m_features), therefore we need to prep the data into this format. Also we need to create our own data generator for model fitting, because 829 samples with the so many features won't fit in the memory once..

1. training / validation / testing dataframe split
2. create generator function to yield encoder_input_x, decoder_input_x, 

### step1 

In [28]:
df_train = df_scaled[(df_scaled['calendar_date'] >= train_enc_start) & (df_scaled['calendar_date'] <= train_pred_end)]
df_validation = df_scaled[(df_scaled['calendar_date'] >= val_enc_start) & (df_scaled['calendar_date'] <= val_pred_end)]
df_test = df_scaled[(df_scaled['calendar_date'] >= test_enc_start) & (df_scaled['calendar_date'] <= test_pred_end)]

In [29]:
df_train.head()

,air_store_id,calendar_date,visitors,holiday_flg,latitude,longitude,day,month,year,dayofweek,dayofyear,lookback_visitors_avg_store,lookback_visitors_avg_genre,lookback_visitors_avg_area,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,avg_wind_speed,avg_humidity,day_after_holiday_flg,day_before_holiday_flg,week_of_month,open_days,lookback_visitors_std_store,lookback_visitors_std_genre,lookback_visitors_std_area,genre_Asian,genre_Bar/Cocktail,genre_Cafe/Sweets,genre_Creative cuisine,genre_Dining bar,genre_International cuisine,genre_Italian/French,genre_Izakaya,genre_Japanese food,genre_Karaoke/Party,genre_Okonomiyaki/Monja/Teppanyaki,genre_Other,genre_Western food,genre_Yakiniku/Korean food,area_Fukuoka-ken Fukuoka-shi Daimyō,area_Fukuoka-ken Fukuoka-shi Hakata Ekimae,area_Fukuoka-ken Fukuoka-shi Imaizumi,area_Fukuoka-ken Fukuoka-shi Momochi,area_Fukuoka-ken Fukuoka-shi Shiobaru,area_Fukuoka-ken Fukuoka-shi Takatori,area_Fukuoka-ken Fukuoka-shi Tenjin,area_Fukuoka-ken Fukuoka-shi Torikai,area_Fukuoka-ken Itoshima-shi Maebarunishi,area_Fukuoka-ken Kitakyūshū-shi Konyamachi,area_Fukuoka-ken Kitakyūshū-shi None,area_Fukuoka-ken Kitakyūshū-shi Ōtemachi,area_Fukuoka-ken Kurume-shi Jōnanmachi,area_Fukuoka-ken Yame-shi Motomachi,area_Hiroshima-ken Fukuyama-shi Higashisakuramachi,area_Hiroshima-ken Hiroshima-shi Kokutaijimachi,area_Hiroshima-ken Hiroshima-shi Nagarekawachō,area_Hokkaidō Abashiri-shi Minami 6 Jōhigashi,area_Hokkaidō Asahikawa-shi 6 Jōdōri,area_Hokkaidō Katō-gun Motomachi,area_Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō,area_Hokkaidō Sapporo-shi Kita 24 Jōnishi,area_Hokkaidō Sapporo-shi Kotoni 2 Jō,area_Hokkaidō Sapporo-shi Minami 3 Jōnishi,area_Hyōgo-ken Amagasaki-shi Higashinanamatsuchō,area_Hyōgo-ken Himeji-shi Yasuda,area_Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,area_Hyōgo-ken Kōbe-shi Kumoidōri,area_Hyōgo-ken Kōbe-shi Motomachidōri,area_Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi,area_Hyōgo-ken Nishinomiya-shi Rokutanjichō,area_Hyōgo-ken Takarazuka-shi Tōyōchō,area_Miyagi-ken Sendai-shi Kamisugi,area_Niigata-ken Kashiwazaki-shi Chūōchō,area_Niigata-ken Nagaoka-shi Ōtedōri,area_Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,area_Niigata-ken Niigata-shi Teraohigashi,area_Shizuoka-ken Hamamatsu-shi Motoshirochō,area_Shizuoka-ken Numazu-shi Miyukichō,area_Shizuoka-ken Shizuoka-shi Ōtemachi,area_Tōkyō-to Adachi-ku Chūōhonchō,area_Tōkyō-to Bunkyō-ku Kasuga,area_Tōkyō-to Bunkyō-ku Sendagi,area_Tōkyō-to Chiyoda-ku Kanda Jinbōchō,area_Tōkyō-to Chiyoda-ku Kudanminami,area_Tōkyō-to Chūō-ku Ginza,area_Tōkyō-to Chūō-ku Tsukiji,area_Tōkyō-to Edogawa-ku Chūō,area_Tōkyō-to Fuchū-shi Miyanishichō,area_Tōkyō-to Itabashi-ku Itabashi,area_Tōkyō-to Katsushika-ku Tateishi,area_Tōkyō-to Kita-ku Ōjihonchō,area_Tōkyō-to Koganei-shi Honchō,area_Tōkyō-to Kōtō-ku Kameido,area_Tōkyō-to Kōtō-ku Tomioka,area_Tōkyō-to Kōtō-ku Tōyō,area_Tōkyō-to Machida-shi Morino,area_Tōkyō-to Meguro-ku Jiyūgaoka,area_Tōkyō-to Meguro-ku Kamimeguro,area_Tōkyō-to Meguro-ku Takaban,area_Tōkyō-to Minato-ku Akasaka,area_Tōkyō-to Minato-ku Roppongi,area_Tōkyō-to Minato-ku Shibakōen,area_Tōkyō-to Musashino-shi Kichijōji Honchō,area_Tōkyō-to Musashino-shi Midorichō,area_Tōkyō-to Nakano-ku Nakano,area_Tōkyō-to Nerima-ku Toyotamakita,area_Tōkyō-to Setagaya-ku Kitazawa,area_Tōkyō-to Setagaya-ku Okusawa,area_Tōkyō-to Setagaya-ku Sangenjaya,area_Tōkyō-to Setagaya-ku Setagaya,area_Tōkyō-to Shibuya-ku Dōgenzaka,area_Tōkyō-to Shibuya-ku Higashi,area_Tōkyō-to Shibuya-ku Jingūmae,area_Tōkyō-to Shibuya-ku Shibuya,area_Tōkyō-to Shinagawa-ku Higashigotanda,area_Tōkyō-to Shinagawa-ku Hiromachi,area_Tōkyō-to Shinjuku-ku Kabukichō,area_Tōkyō-to Shinjuku-ku Nishishinjuku,area_Tōkyō-to Suginami-ku Asagayaminami,area_Tōkyō-to Tachikawa-shi Izumichō,area_Tōkyō-to Taitō-ku Asakusa,area_Tōkyō-to Taitō-ku Higashiueno,area_Tōkyō-to Taitō-ku Kuramae,area_Tōkyō-to Toshima-ku Higashiikebukuro,area_Tōkyō-to Toshima-ku Mejiro,area_Tōkyō-to Toshima-ku Minamiikebukuro,area_Tō

In [30]:
df_train.tail()

,air_store_id,calendar_date,visitors,holiday_flg,latitude,longitude,day,month,year,dayofweek,dayofyear,lookback_visitors_avg_store,lookback_visitors_avg_genre,lookback_visitors_avg_area,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,avg_wind_speed,avg_humidity,day_after_holiday_flg,day_before_holiday_flg,week_of_month,open_days,lookback_visitors_std_store,lookback_visitors_std_genre,lookback_visitors_std_area,genre_Asian,genre_Bar/Cocktail,genre_Cafe/Sweets,genre_Creative cuisine,genre_Dining bar,genre_International cuisine,genre_Italian/French,genre_Izakaya,genre_Japanese food,genre_Karaoke/Party,genre_Okonomiyaki/Monja/Teppanyaki,genre_Other,genre_Western food,genre_Yakiniku/Korean food,area_Fukuoka-ken Fukuoka-shi Daimyō,area_Fukuoka-ken Fukuoka-shi Hakata Ekimae,area_Fukuoka-ken Fukuoka-shi Imaizumi,area_Fukuoka-ken Fukuoka-shi Momochi,area_Fukuoka-ken Fukuoka-shi Shiobaru,area_Fukuoka-ken Fukuoka-shi Takatori,area_Fukuoka-ken Fukuoka-shi Tenjin,area_Fukuoka-ken Fukuoka-shi Torikai,area_Fukuoka-ken Itoshima-shi Maebarunishi,area_Fukuoka-ken Kitakyūshū-shi Konyamachi,area_Fukuoka-ken Kitakyūshū-shi None,area_Fukuoka-ken Kitakyūshū-shi Ōtemachi,area_Fukuoka-ken Kurume-shi Jōnanmachi,area_Fukuoka-ken Yame-shi Motomachi,area_Hiroshima-ken Fukuyama-shi Higashisakuramachi,area_Hiroshima-ken Hiroshima-shi Kokutaijimachi,area_Hiroshima-ken Hiroshima-shi Nagarekawachō,area_Hokkaidō Abashiri-shi Minami 6 Jōhigashi,area_Hokkaidō Asahikawa-shi 6 Jōdōri,area_Hokkaidō Katō-gun Motomachi,area_Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō,area_Hokkaidō Sapporo-shi Kita 24 Jōnishi,area_Hokkaidō Sapporo-shi Kotoni 2 Jō,area_Hokkaidō Sapporo-shi Minami 3 Jōnishi,area_Hyōgo-ken Amagasaki-shi Higashinanamatsuchō,area_Hyōgo-ken Himeji-shi Yasuda,area_Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,area_Hyōgo-ken Kōbe-shi Kumoidōri,area_Hyōgo-ken Kōbe-shi Motomachidōri,area_Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi,area_Hyōgo-ken Nishinomiya-shi Rokutanjichō,area_Hyōgo-ken Takarazuka-shi Tōyōchō,area_Miyagi-ken Sendai-shi Kamisugi,area_Niigata-ken Kashiwazaki-shi Chūōchō,area_Niigata-ken Nagaoka-shi Ōtedōri,area_Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,area_Niigata-ken Niigata-shi Teraohigashi,area_Shizuoka-ken Hamamatsu-shi Motoshirochō,area_Shizuoka-ken Numazu-shi Miyukichō,area_Shizuoka-ken Shizuoka-shi Ōtemachi,area_Tōkyō-to Adachi-ku Chūōhonchō,area_Tōkyō-to Bunkyō-ku Kasuga,area_Tōkyō-to Bunkyō-ku Sendagi,area_Tōkyō-to Chiyoda-ku Kanda Jinbōchō,area_Tōkyō-to Chiyoda-ku Kudanminami,area_Tōkyō-to Chūō-ku Ginza,area_Tōkyō-to Chūō-ku Tsukiji,area_Tōkyō-to Edogawa-ku Chūō,area_Tōkyō-to Fuchū-shi Miyanishichō,area_Tōkyō-to Itabashi-ku Itabashi,area_Tōkyō-to Katsushika-ku Tateishi,area_Tōkyō-to Kita-ku Ōjihonchō,area_Tōkyō-to Koganei-shi Honchō,area_Tōkyō-to Kōtō-ku Kameido,area_Tōkyō-to Kōtō-ku Tomioka,area_Tōkyō-to Kōtō-ku Tōyō,area_Tōkyō-to Machida-shi Morino,area_Tōkyō-to Meguro-ku Jiyūgaoka,area_Tōkyō-to Meguro-ku Kamimeguro,area_Tōkyō-to Meguro-ku Takaban,area_Tōkyō-to Minato-ku Akasaka,area_Tōkyō-to Minato-ku Roppongi,area_Tōkyō-to Minato-ku Shibakōen,area_Tōkyō-to Musashino-shi Kichijōji Honchō,area_Tōkyō-to Musashino-shi Midorichō,area_Tōkyō-to Nakano-ku Nakano,area_Tōkyō-to Nerima-ku Toyotamakita,area_Tōkyō-to Setagaya-ku Kitazawa,area_Tōkyō-to Setagaya-ku Okusawa,area_Tōkyō-to Setagaya-ku Sangenjaya,area_Tōkyō-to Setagaya-ku Setagaya,area_Tōkyō-to Shibuya-ku Dōgenzaka,area_Tōkyō-to Shibuya-ku Higashi,area_Tōkyō-to Shibuya-ku Jingūmae,area_Tōkyō-to Shibuya-ku Shibuya,area_Tōkyō-to Shinagawa-ku Higashigotanda,area_Tōkyō-to Shinagawa-ku Hiromachi,area_Tōkyō-to Shinjuku-ku Kabukichō,area_Tōkyō-to Shinjuku-ku Nishishinjuku,area_Tōkyō-to Suginami-ku Asagayaminami,area_Tōkyō-to Tachikawa-shi Izumichō,area_Tōkyō-to Taitō-ku Asakusa,area_Tōkyō-to Taitō-ku Higashiueno,area_Tōkyō-to Taitō-ku Kuramae,area_Tōkyō-to Toshima-ku Higashiikebukuro,area_Tōkyō-to Toshima-ku Mejiro,area_Tōkyō-to Toshima-ku Minamiikebukuro,area_Tō

### step2

In [31]:
# Tensorflow only accept numpy array as input so we need to convert the format 
# and also reshape input data(all the numeric features) and target output(visitors) it into(n_sampels, t_timesteps, m_features)
def df_to_array(df_data): #
    y_index = df_data.columns.get_loc("visitors")
    #Y_array = df_data.values[:,y_index]\
    #                 .reshape(df_data.air_store_id.nunique(), df_data.calendar_date.nunique(), 1)
    #X_array = df_data.values[:,(y_index + 1):]\
    #                 .reshape(df_data.air_store_id.nunique(), df_data.calendar_date.nunique(), df_data.shape[1]-3)
    
    #return X_array, Y_array
    X_y_array = df_data.values[:,y_index:]\
                     .reshape(df_data.air_store_id.nunique(), df_data.calendar_date.nunique(), -1)
    # the first elements in 3rd dimension is visitors - target variable, and are following with other features
    
    store_date = df_data.iloc[:,:y_index]
    return X_y_array, store_date

In [32]:
train_array, _ = df_to_array(df_train)
validation_array, val_store_date_df = df_to_array(df_validation)
test_array, store_date_df = df_to_array(df_test)

In [33]:
NONSTATIONARY_INDEX = df_scaled.iloc[:,2:].columns.get_loc("closed_flag") # all the features following closed flag is nonstationary
VISITORS_INDEX = df_scaled.iloc[:,2:].columns.get_loc("visitors") # target value

In [34]:
NONSTATIONARY_INDEX

143

In [35]:
# create a random data generator, each iteration it will generator
# encoding input (batch_size stores, 400 days, m_features )
# decoding input (batch_size stores, 39 days, # of stationary geatures)
# decoder target (batch_size stores, 39 days, 1)
def enc_dec_data_generator(data, batch_size): 
    while 1:
        n_samples = data.shape[0]
        if batch_size <= n_samples:
            # randomly select batchsize samples out of data
            batch_samples = random.sample(range(0, n_samples), batch_size) 
            selected_data = data[batch_samples]
            
            # we need to delete those selected samples, so that we won't sample duplicates
            mask = np.ones(n_samples, bool) 
            mask[batch_samples]=False
            data = data[mask]
        else:
            # if the remaining samples are less than the batch_size we will select all the remaining data
            selected_data = data 
        
        # now Let's tease out the selected data into X_enc_in, X_dec_in, Y_dec_out
        #print(selected_data.shape)
        X_enc_in = selected_data[:, :LOOKBACK_DAYS, :] # we keep all features including visitors number in lookback periods, 
        X_dec_in = selected_data[:, LOOKBACK_DAYS:, VISITORS_INDEX + 1 : NONSTATIONARY_INDEX] # only keep stationary features in prediction periods
        Y_dec_out = selected_data[:, LOOKBACK_DAYS:, VISITORS_INDEX].reshape(-1, PRED_DAYS, 1) # only keep visitors as target value in prediction periods
        #print(Y_dec_out.shape)
        #print(X_enc_in.shape)
        #print(X_dec_in)
        yield ([X_enc_in, X_dec_in], Y_dec_out)
        #.reshape(n_samples, PRED_DAYS,1)
              

In [36]:
# test if this self defined generaotr works
#next(train_generator)

# Build Model

Model Structure:

We will use past 400 days data to predict visitors traffic of following 39 days
The encoder would consume information including visitors, reservation info, weather info, holiday info
Decoder would also be fed with future 39 days data with stationary/certain information eg. holiday info, day of week etc.  to predict visitors as output

We hope that the model can pick up latent pattern based on past 400 days data and with the aids of future 39 days info, the boy can predict decent outcome for us. So let's get do it.

Model code referenced from [https://towardsdatascience.com/time-series-forecasting-with-deep-stacked-unidirectional-and-bidirectional-lstms-de7c099bd918]

In [37]:
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense, LSTMCell, RNN, Bidirectional, concatenate
from keras.optimizers import Adam
from keras.layers import TimeDistributed

Using TensorFlow backend.


In [38]:
def create_model(n_enc_in_features, n_dec_in_features, n_dec_out_features, layers, activation = 'tanh', optimizer = 'adam', dropout_rate = 0.5, bidirectional=False):
    
    n_layers = len(layers)
    
    ## Encoder
    encoder_inputs = Input(shape=(None, n_enc_in_features))
    lstm_cells = [LSTMCell(hidden_dim, activation = activation, dropout = dropout_rate) for hidden_dim in layers]
    if bidirectional:
        encoder = Bidirectional(RNN(lstm_cells, return_state=True))
        encoder_outputs_and_states = encoder(encoder_inputs)
        bi_encoder_states = encoder_outputs_and_states[1:]
        encoder_states = []
        for i in range(int(len(bi_encoder_states)/2)):
            temp = concatenate([bi_encoder_states[i],bi_encoder_states[2*n_layers + i]], axis=-1)
            encoder_states.append(temp)
    else:  
        encoder = RNN(lstm_cells, return_state=True, name = 'encoder')
        encoder_outputs_and_states = encoder(encoder_inputs)
        encoder_states = encoder_outputs_and_states[1:]
    
    ## Decoder
    decoder_inputs = Input(shape=(None, n_dec_in_features))
    
    if bidirectional:
        decoder_cells = [LSTMCell(hidden_dim*2) for hidden_dim in layers]
    else:
        decoder_cells = [LSTMCell(hidden_dim, activation = activation, dropout = dropout_rate) for hidden_dim in layers]
        
    decoder_lstm = RNN(decoder_cells, return_sequences=True, return_state=True, name = 'decoder')
    decoder_outputs_and_states = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_outputs = decoder_outputs_and_states[0]
    decoder_dense1 = Dense(100, activation = 'linear')
    decoder_outputs = decoder_dense1(decoder_outputs)
    decoder_dense2 = TimeDistributed(Dense(1, activation = 'linear'))
    #decoder_dense2 = Dense(n_dec_out_features, activation = 'linear') 
    decoder_outputs = decoder_dense2(decoder_outputs)
    
    model = Model([encoder_inputs,decoder_inputs], decoder_outputs)
    return model


In [39]:
df_train.columns.shape
df_train.columns

Index(['air_store_id', 'calendar_date', 'visitors', 'holiday_flg', 'latitude',
       'longitude', 'day', 'month', 'year', 'dayofweek', 'dayofyear',
       'lookback_visitors_avg_store', 'lookback_visitors_avg_genre',
       'lookback_visitors_avg_area', 'avg_temperature', 'high_temperature',
       'low_temperature', 'precipitation', 'hours_sunlight', 'avg_wind_speed',
       'avg_humidity', 'day_after_holiday_flg', 'day_before_holiday_flg',
       'week_of_month', 'open_days', 'lookback_visitors_std_store',
       'lookback_visitors_std_genre', 'lookback_visitors_std_area',
       'genre_Asian', 'genre_Bar/Cocktail', 'genre_Cafe/Sweets',
       'genre_Creative cuisine', 'genre_Dining bar',
       'genre_International cuisine', 'genre_Italian/French', 'genre_Izakaya',
       'genre_Japanese food', 'genre_Karaoke/Party',
       'genre_Okonomiyaki/Monja/Teppanyaki', 'genre_Other',
       'genre_Western food', 'genre_Yakiniku/Korean food',
       'area_Fukuoka-ken Fukuoka-shi Daimyō',
  

In [40]:
N_enc_in_features = 146
N_dec_in_features = 142
N_dec_out_features = 1

In [41]:
#We pass the hidden dimensions in each layer in the form of a list. 
#so length of the list becomes depth of the network.  
#model = create_model([60,60], bidirectional = False)  

#Let's start from simple 1 layer structure, with 256 hidden layer
model = create_model( N_enc_in_features, N_dec_in_features, N_dec_out_features, layers = [256],bidirectional = False) 

In [42]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 146)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 142)    0                                            
__________________________________________________________________________________________________
encoder (RNN)                   [(None, 256), (None, 412672      input_1[0][0]                    
__________________________________________________________________________________________________
decoder (RNN)                   [(None, None, 256),  408576      input_2[0][0]                    
                                                                 encoder[0][1]              

In [43]:
BATCH_SIZE = 256 # each time we train on only 256 stores 439 days records
N_TRIANING_INSTANCE = 829
STEP_SIZE=N_TRIANING_INSTANCE//BATCH_SIZE # number of training instances // each batch size
EPOCH = 15

# Hyperparameters Tunning

1. Optimizers and learning rate
2. Model Structure, we will try more neurons and deeper layers
3. activation function
4. dropout rate

In [44]:
import keras
learning_rates = [1e-5, 1e-4,1e-3]
optimizers = ['adam', 'rmsprop', 'adagrad']
for o in optimizers:
    for i in learning_rates:
        print('----------optimizers-', o)
        print('learning rate: ', i)
        model = create_model(N_enc_in_features, N_dec_in_features, N_dec_out_features, layers=[256], bidirectional = False) 

        if o == 'adam':
            optimize_instance = keras.optimizers.Adam(learning_rate = i, beta_1=0.9, beta_2=0.999, amsgrad=False)
        elif o == 'rmsprop':
            optimize_instance = keras.optimizers.RMSprop(learning_rate= i, rho=0.9, decay = 1e-6)
        elif o == 'adagrad':
            optimize_instance = keras.optimizers.Adagrad(learning_rate= i)

        model.compile(optimize_instance, loss = 'mean_squared_error')
        train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
        validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
        model.fit_generator(generator = train_generator,
                            steps_per_epoch= STEP_SIZE, 
                            # Total number of steps (batches of samples) to yield from generator 
                            # before declaring one epoch finished and starting the next epoch.
                            validation_data=validation_generator,
                            validation_steps = STEP_SIZE,
                            epochs=15)

----------optimizers- adam
learning rate:  1e-05
Epoch 1/15
2/3 [===================>..........] - ETA: 7s - loss: 5.5294 

KeyboardInterrupt: 

optimizer candidate: adam with 0.001 learning rate

## After we decided on the optimizer and learning rate, Let's search on other Hyperparameters

In [ ]:
layers = [[512],[1024],[256,256],[512,512]]
activations = ['tanh']
optimizers = ['adam']
dropoutrates = [0.5, 0.3]

In [ ]:
param_grid = dict(
            activation = activations, 
            optimizer = optimizers, 
            dropout_rate = dropoutrates,
            layers = layers
)
import itertools as it
param_combinations = [dict(zip(param_grid,v)) for v in it.product(*param_grid.values())]

In [ ]:
param_combinations

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
# patience = 3 means that if the accuracy on test set does not improve over 3 epochs we will stop the training

In [ ]:
# Now Let's do a simulated GridSearch and keep track of the result in a dataframe
rows = [] 
for i in param_combinations:
    o = i['optimizer']
    model = create_model(N_enc_in_features, N_dec_in_features, N_dec_out_features, **i, bidirectional = False) 

    if o == 'adam':
        optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    elif o == 'rmsprop':
        optimize_instance = keras.optimizers.RMSprop(learning_rate= 0.001, rho=0.9, decay = 1e-6)
    elif o == 'adagrad':
        optimize_instance = keras.optimizers.Adagrad(learning_rate= 0.001)

    model.compile(optimize_instance, loss = 'mean_squared_error')
    train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
    validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
    history = model.fit_generator(generator = train_generator,
                        steps_per_epoch= STEP_SIZE, 
                        # Total number of steps (batches of samples) to yield from generator 
                        # before declaring one epoch finished and starting the next epoch.
                        validation_data=validation_generator,
                        validation_steps = STEP_SIZE,
                        epochs=EPOCH,
                        callbacks=[es]
                                 )
    
    i['loss'] = history.history['loss']
    i['loss_last'] = history.history['loss'][-1]
    i['loss_std'] = np.array(history.history['loss']).std()
    
    i['val_loss'] = history.history['val_loss']
    i['val_loss_last'] = history.history['val_loss'][-1]
    i['val_loss_std'] = np.array(history.history['val_loss']).std()
    
    rows.append(i)

GridSearch_result = pd.DataFrame.from_dict(rows, orient='columns')

In [ ]:
GridSearch_result

# Final Pick
[256,256] 2 layers of LSTM cells structure, default 'tanh' activation for LSTM cell, dropout rate of 0.5, using Adam as optimizers with learning rate of 0.001

In [ ]:
layers = [[256,256]]
activations = ['tanh']
optimizers = ['adam']
dropoutrates = [0.5]
param_grid = dict(
            activation = activations, 
            optimizer = optimizers, 
            dropout_rate = dropoutrates,
            layers = layers
)
import itertools as it
param_combinations = [dict(zip(param_grid,v)) for v in it.product(*param_grid.values())]

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 20)
# patience = 3 means that if the accuracy on test set does not improve over 10 epochs we will stop the training

In [ ]:
for i in param_combinations:
    o = i['optimizer']
    model = create_model(N_enc_in_features, N_dec_in_features, N_dec_out_features, **i, bidirectional = False) 

    if o == 'adam':
        optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    elif o == 'rmsprop':
        optimize_instance = keras.optimizers.RMSprop(learning_rate= 0.001, rho=0.9, decay = 1e-6)
    elif o == 'adagrad':
        optimize_instance = keras.optimizers.Adagrad(learning_rate= 0.001)

    model.compile(optimize_instance, loss = 'mean_squared_error')
    train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
    validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
    history = model.fit_generator(generator = train_generator,
                        steps_per_epoch= STEP_SIZE, 
                        # Total number of steps (batches of samples) to yield from generator 
                        # before declaring one epoch finished and starting the next epoch.
                        validation_data=validation_generator,
                        validation_steps = STEP_SIZE,
                        epochs=100,
                        callbacks=[es]
                                 )

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot traning histoy -- loss
plt.plot(history.history['loss'], label = 'train')
plt.plot(history.history['val_loss'], label = 'validation')
plt.title('MSLE loss')
plt.xlabel('epochs')
plt.legend()
plt.show()

In [ ]:
model.summary()

In [ ]:
model.save('./model2.hdf5')

## Check out the true RMSE error on validation set

In [45]:
from keras.models import load_model
model = load_model('model2.hdf5')

In [77]:
y = validation_array[:, LOOKBACK_DAYS:, :VISITORS_INDEX + 1]
x_enc = validation_array[:, :LOOKBACK_DAYS, :]
x_dec = validation_array[:, LOOKBACK_DAYS:, VISITORS_INDEX +1: NONSTATIONARY_INDEX]

In [78]:
predicted_array = model.predict([x_enc,x_dec]).reshape(829,39)

In [79]:
target_array = y.reshape(829,39)# 829 stores in validation sets

The model give us no meaningful output..

In [83]:
 np.e**target_array - 1

array([[41.99999999999999, 38.99999999999999, 36.999999999999986, ...,
        37.999999999999986, 55.000000000000014, 17.999999999999993],
       [10.999999999999998, 4.999999999999999, 14.999999999999996, ...,
        0.9999999999999998, 25.999999999999996, 5.999999999999999],
       [6.999999999999997, 0.0, 5.999999999999999, ..., 0.0,
        2.9999999999999996, 12.999999999999995],
       ...,
       [12.999999999999995, 4.999999999999999, 4.999999999999999, ...,
        0.9999999999999998, 2.9999999999999996, 4.999999999999999],
       [18.999999999999993, 26.999999999999993, 29.999999999999996, ...,
        23.0, 54.0, 0.9999999999999998],
       [6.999999999999997, 2.9999999999999996, 8.0, ..., 2.0,
        3.999999999999999, 4.999999999999999]], dtype=object)

In [84]:
np.e**predicted_array - 1

array([[11.602809 ,  6.4094014,  8.042451 , ..., 11.520981 , 11.371147 ,
        11.270796 ],
       [10.760584 ,  6.043376 ,  7.4984713, ..., 10.7905655, 10.678283 ,
        10.615391 ],
       [14.385886 ,  7.7427206,  9.956316 , ..., 15.531088 , 15.370125 ,
        15.363842 ],
       ...,
       [13.257546 ,  7.035532 ,  8.664517 , ..., 13.110338 , 13.2529545,
        13.731865 ],
       [12.373196 ,  6.908324 ,  8.753207 , ..., 13.541771 , 13.379206 ,
        13.286047 ],
       [12.440714 ,  6.684058 ,  8.210319 , ..., 12.053259 , 11.859144 ,
        11.742332 ]], dtype=float32)

In [70]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
# because we made prediction on log(1+visitors), so this function will gave us root mean squared log error
error = mean_squared_error(target_array, predicted_array)
print('root mean squared log error is', np.sqrt(error))

root mean squared log error is 1.2396431610728091


In [71]:
# we need to transform the value back to the original scale to calculate mean square error
target_array_reversed = np.e**target_array - 1
predicted_array_reversed = np.e**predicted_array - 1
error = mean_squared_error(target_array_reversed, predicted_array_reversed)
print('root mean squared error is', np.sqrt(error))

root mean squared error is 20.097154022892838


In [55]:
# R2 scores
from sklearn.metrics import r2_score
print('R2: ', r2_score(target_array_reversed, predicted_array_reversed))

R2:  -0.20102823033859424


In [51]:
# double check if the RMSLE is right
target_array_reversed = np.e**target_array - 1
predicted_array_reversed = np.e**predicted_array - 1
error = mean_squared_log_error(target_array_reversed, predicted_array_reversed)
print('root mean squared log error is', np.sqrt(error))

root mean squared error is 1.2396431608456737


In [85]:
predicted_array = model.predict([x_enc,x_dec]).reshape(-1)

In [86]:
val_output = val_store_date_df.loc[val_store_date_df['calendar_date'] >= '2017-03-15',:]

In [87]:
val_output['visitors']=np.e**predicted_array-1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [88]:
val_output.to_csv('val_output.csv', index = False)

# Final Prediction on submission sample file

In [89]:
#y = test_array[:, LOOKBACK_DAYS:, :VISITORS_INDEX + 1]
x_enc = test_array[:, :LOOKBACK_DAYS, :]
x_dec = test_array[:, LOOKBACK_DAYS:, VISITORS_INDEX +1: NONSTATIONARY_INDEX]

In [90]:
predicted_array = model.predict([x_enc,x_dec]).reshape(-1)

In [91]:
test_output = store_date_df.loc[store_date_df['calendar_date'] >= '2017-04-23',:]

In [92]:
predicted_array.shape

(32331,)

In [93]:
test_output.shape

(32331, 2)

In [94]:
test_output['visitors']=np.e**predicted_array-1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [95]:
test_output

,air_store_id,calendar_date,visitors
478,air_00a91d42b08b08d9,2017-04-23,13.500292
479,air_00a91d42b08b08d9,2017-04-24,7.034383
480,air_00a91d42b08b08d9,2017-04-25,9.136913
481,air_00a91d42b08b08d9,2017-04-26,11.156055
482,air_00a91d42b08b08d9,2017-04-27,12.343884
...,...,...,...
428588,air_fff68b929994bfbd,2017-05-27,13.439842
428589,air_fff68b929994bfbd,2017-05-28,13.567477
428590,air_fff68b929994bfbd,2017-05-29,12.656897
428591,air_fff68b929994bfbd,2017-05-30,13.524006


In [ ]:
test_output.to_csv('output1.csv', index = False)

In [ ]:
test_output['calendar_date'] = pd.to_datetime(test_output['calendar_date'])
samp_sub['calendar_date'] = pd.to_datetime(samp_sub['calendar_date'])

In [ ]:
test_output.dtypes

In [ ]:
samp_sub.dtypes

In [ ]:
samp_sub_output = samp_sub.merge(test_output, on = ['air_store_id','calendar_date'],how ='left')

In [ ]:
samp_sub_output.to_csv('samp_sub_output.csv', index = False)